In [2]:
import os
import numpy as np
import torchvision
import torch
import cv2
from tqdm import tqdm

/Users/sreenathswaminathan/Desktop/Uni-Docs/Autonomous fahren kurs/CV Project/Pytorch_Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [29]:
# from io import StringIO, BytesIO, TextIOWrapper
from zipfile import ZipFile 

# dataset = 'kagglewolvessanddogs_5340.zip'
# zipfile = ZipFile(BytesIO(dataset.read()))
with ZipFile('archive.zip','r') as zip_ref:
    zip_ref.extractall('UpdatedDataset')


In [1]:
REBUILD_DATA = True

training_data =[]

class DogsVsWolves():
    IMG_SIZE = 60
    Dogs = "UpdatedDataset/data/dogs"
    Wolves = "UpdatedDataset/data/wolves"
    LABELS = {Wolves: 0, Dogs: 1}

    
    Dog_count = 0
    Wolves_count = 0

    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:    
                    path = os.path.join(label,f)
                    # print(path)

                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

                    try:
                        img = cv2.resize(img,(self.IMG_SIZE, self.IMG_SIZE))
                    except cv2.error as e:
                        # print("Configure error")
                        continue

                    label_encoded = np.eye(2)[self.LABELS[label]]
                    
                    # Ensure img is a 2D array
                    img = np.array(img)
                    
                    
                    try:
                        training_data.append([img, label_encoded])
                        # print(self.training_data)
                    except cv2.error as e:
                        pass

                    if label == self.Wolves:
                        self.Wolves_count += 1
                    elif label == self.Dogs:
                        self.Dog_count += 1

                except Exception as e:
                    # print(f"Error loading image: {path} - {str(e)}")
                    pass

        np.random.shuffle(training_data)
        # print(self.training_data)
        np.save("training_data.npy", training_data)
        print("Wolves", self.Wolves_count)
        print("Dogs", self.Dog_count)


if REBUILD_DATA:
    DogsVsWolves = DogsVsWolves()
    DogsVsWolves.make_training_data()


UpdatedDataset/data/wolves


NameError: name 'tqdm' is not defined

In [52]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

EOFError: No data left in file

In [58]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64,128,5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear,512)
        self.fc2 = nn.Linear(512,2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        print(x.shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)
    
net = Net()

torch.Size([1, 128, 2, 2])


In [55]:
net

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [59]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr =0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,60,60)
X = X/255.0
Y = torch.Tensor([i[0] for i in training_data])

VAL_PCT = 0.1 # Percentage of the validation dataset from the total x
val_size = int(len(X)*VAL_PCT)
print(val_size)

NameError: name 'training_data' is not defined

In [60]:
train_X = X[:-val_size]
train_Y = Y[:-val_size]

test_X = X[-val_size:]
test_Y = Y[-val_size:]

NameError: name 'X' is not defined

In [65]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0,len(train_X), BATCH_SIZE)):
        # print(i, i+BATCH_SIZE)
        batch_x = train_X[i:i+BATCH_SIZE].view(-1,1,60,60)
        batch_y = train_Y[i,i+BATCH_SIZE]

        optimizer.zero_grad()
        outputs = net(batch_x)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

print(loss)

NameError: name 'train_X' is not defined

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_Y[i])
        net_out = net(test_X[i].view(-1,1,60,60))[0]

        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy:",round(correct/total,3))